In [1]:
from collections import defaultdict
import json
import os

import requests
import ricecooker
from bs4 import BeautifulSoup

from tessa_chef import get_modtype, get_resource_info
from tessa_chef import process_language_page
from tessa_chef import get_text, create_subpage_node
from tessa_chef import  TESSA_LANG_URL_MAP, TESSA_HOME_URL

from tessa_chef import DATA_DIR, TREES_DATA_DIR, CRAWLING_STAGE_OUTPUT, SCRAPING_STAGE_OUTPUT

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)

In [2]:
from tessa_chef import TessaChef
tessa_chef = TessaChef()

## Explore crawling with the following debugging steps

In [3]:
tessa_chef.crawl(None, {'lang':'all'})




------- JULY 28 SPECIAL OUTPUT FOR CRAWLING REVIEW ------
crawling lang= en starting at http://www.open.edu/openlearncreate/course/view.php?id=2042



Processing TESSA page  EN     num of unfiltered activity links: 22
Append description: The subject resources are divi..


http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66754
 - Recognizd standard module structure. Taking whole module:
   Content (oucontent): Module 1: Personal development – how self-esteem impacts on learning
    - skipping module section li Section 1: Ways to explore who pupils are Unit content
    - skipping module section li Section 2: Planning physical growth and development sessions Unit content
    - skipping module section li Section 3: Exploring pupils' ideas about healthy living Unit content
    - skipping module section li Section 4: Activities to support emotional well-being Unit content
    - skipping module section li Section 5: Ways to promote spiritual well-being Unit content
    - skippin

 - Recognizd standard module structure. Taking whole module:
   Content (oucontent): Module 1: Science - Looking at Life
    - skipping module section li Section 1: Classifying living things Unit content
    - skipping module section li Section 2: A closer look at plants Unit content
    - skipping module section li Section 3: Investigating animals: hunters and the hunted Unit content
    - skipping module section li Section 4: Plants and animals adapting to survive Unit content
    - skipping module section li Section 5: Developing attitudes about our environmental impact Unit content
    - skipping other li Read or download individual sections of 
 - Recognizd standard module structure. Taking whole module:
   Content (oucontent): Module 2: Science - Investigating Materials
    - skipping module section li Section 1: Investigating and classifying materials Unit content
    - skipping module section li Section 2: Exploring solids Unit content
    - skipping module section li Section 3




Processing TESSA page  FR     num of unfiltered activity links: 16
Ignoring activity of type resource 	startswith: Récapitulatif détaillé TESSA


http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66857
       - Nonstandard module, using fallback strategies...
         section_li (label) Module 1 : Développement personnel – Imp
         section_li (oucontent) Module 1 : Développement personnel – Imp
         section_li (label) Lisez ou téléchargez des sections spécif
         section_li (oucontent) Section numéro 1 : Comment mieux connaît
         section_li (oucontent) Section numéro 2 : Préparation de séance
         section_li (oucontent) Section numéro 3 : Exploration des idées
         section_li (oucontent) Section numéro 4 : Promotion du bien-êtr
         section_li (oucontent) Section numéro 5 : Promotion du bien-êtr
         section_li (label) Module 2 : Exploration du développement 
         section_li (oucontent) Module 2 : Exploration du développement 
         

       - Nonstandard module, using fallback strategies...
         section_li (label) Les ressources clés ont été produites po
         section_li (oucontent) Ressources clés pour les enseignants et 
         section_li (label) Pour lire ou télécharger une ressource i
         section_li (oucontent) Ressource clé : Travailler avec des clas
         section_li (oucontent) Ressource clé : Travailler en groupes da
         section_li (oucontent) Ressource clé : Utiliser des cartes conc
         section_li (oucontent) Ressource clé : Planifiez et préparez vo
         section_li (oucontent) Ressource clé : Evaluer l'apprentissage 
         section_li (oucontent) Ressource clé : Travail d'investigation 
         section_li (oucontent) Ressource clé : Être chercheur dans sa c
         section_li (oucontent) Ressource clé : Utiliser les jeux de rôl
         section_li (oucontent) Ressource clé : Utiliser le questionneme
         section_li (oucontent) Ressource clé : Utiliser les récits dans
 

    - skipping other li قم بقراءة أو تنزيل كل قسم من أقسام الوحد
    - skipping other li عودة إلى القائمة الرئيسيةTESSA عودة إلى 


http://www.open.edu/openlearncreate/mod/subpage/view.php?id=81257
       - Nonstandard module, using fallback strategies...
         section_li (label) وجد 15 وحدة لمادة العلوم للمرحلة الثانوي
         section_li (label) ونظراً لأن هذه الموارد متاحة باللغة الإن
         section_li (label) الأحياء
         section_li (url) Module 1: Secondary Science - Biology UR
         section_li (label) قم بقراءة أو تنزيل كل قسم من أقسام الوحد
         section_li (url) Section 1 : Classification and adaptatio
         section_li (url) Section 2 : Transport URL
         section_li (url) Section 3 : Respiration URL
         section_li (url) Section 4 : Nutrition, conservation and 
         section_li (url) Section 5 : Cells URL
         section_li (label) الكيمياء
         section_li (url) Module 2: Secondary Science - Chemistry 
         section_li (label) قم بقراءة أو ت

    - skipping module section li Sehemu ya 5: Mazoezi ya hesabu za sehemu Unit content
    - skipping other li Namba ya moduli 1:Kuchunguza namba na su
    - skipping other li Soma au pakua vipengele binafsi vya modu
 - Recognizd standard module structure. Taking whole module:
   Content (oucontent): Namba ya moduli 2: Kuchunguza Maumbo na Uwazi
    - skipping module section li Sehemu ya 1: Kuchunguza maumbo Unit content
    - skipping module section li Sehemu ya 2: Njia za utendaji toka katika karatasi hadi utengenezaji wa mchemraba Unit content
    - skipping module section li Sehemu ya 3: Kuchunguza maumbo yenye pande 3 Unit content
    - skipping module section li Sehemu ya 4: Kuchunguza ulinganifu Unit content
    - skipping module section li Sehemu ya 5: Kufundisha Ugeuzi Unit content
    - skipping other li Namba ya moduli 2: Kuchunguza Maumbo na 
    - skipping other li Soma au pakua vipengele binafsi vya modu
 - Recognizd standard module structure. Taking whole module:
   Cont

In [4]:
with open(os.path.join(TREES_DATA_DIR, CRAWLING_STAGE_OUTPUT.replace('.json', '_unfiltered.json'))) as json_file:
    web_resource_trees = json.load(json_file)

for web_resource_tree in web_resource_trees:
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        print('           desciption:', category['description'][0:60]+'..') # len(category['description']))
        
        for resource in category['children']:
            # print(resource)
            if 'kind' not in resource:
                resource['kind'] = resource['type']
            print('      - Resource (%s):' % resource['kind'], resource['title'])
            for child in resource['children']:
                # print(child)
                print('         - Child (%s):' % child['kind'], child['title'])
                
    print('\n\n')



----
WEB RESOURCE TREE: title: TESSA (EN)   len(children) = 4
      description: TESSA materials have been created and developed by experts w..
   - Category   title: Curriculum framework
           desciption: An outline of all the TESSA modules can be found in the foll..
      - Resource (oucontent): Primary Curriculum framework
      - Resource (oucontent): Secondary Science Curriculum framework
   - Category   title: Subject resources
           desciption:  The subject resources are divided into six subject areas wh..
      - Resource (TessaSubpage): Life Skills (primary)
         - Child (TessaModule): Module 1: Personal development – how self-esteem impacts on learning
         - Child (TessaModule): Module 2: Exploring social development
         - Child (TessaModule): Module 3: Community issues and citizenship
      - Resource (TessaSubpage): Literacy (primary)
         - Child (TessaModule): Module 1: Reading and writing for a range of purposes
         - Child (TessaModule):

In [5]:
from tessa_chef import get_parsed_html_from_url


for web_resource_tree in filtered_tessa_wrtrees:
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        print('           desciption:', category['description'][0:60]+'..') # len(category['description']))
        
        for resource in category['children']:
            # print(resource)
            print('      - Resource (%s):' % resource['type'], resource['title'])
            for child in resource['children']:
                # print(child)
                doc = get_parsed_html_from_url(child['url'])
                if has_html_zip(doc):
                    print('         - Child (%s):' % child['type'], child['title'], child['url'])
                else:
                    print(' NOHTMLZIP Child (%s):' % child['type'], child['title'], child['url'])


NameError: name 'filtered_tessa_wrtrees' is not defined